# Assignment 4

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

This assignment requires that you to find **at least** two datasets on the web which are related, and that you visualize these datasets to answer a question with the broad topic of **economic activity or measures** (see below) for the region of **Ann Arbor, Michigan, United States**, or **United States** more broadly.

You can merge these datasets with data from different regions if you like! For instance, you might want to compare **Ann Arbor, Michigan, United States** to Ann Arbor, USA. In that case at least one source file must be about **Ann Arbor, Michigan, United States**.

You are welcome to choose datasets at your discretion, but keep in mind **they will be shared with your peers**, so choose appropriate datasets. Sensitive, confidential, illicit, and proprietary materials are not good choices for datasets for this assignment. You are welcome to upload datasets of your own as well, and link to them using a third party repository such as github, bitbucket, pastebin, etc. Please be aware of the Coursera terms of service with respect to intellectual property.

Also, you are welcome to preserve data in its original language, but for the purposes of grading you should provide english translations. You are welcome to provide multiple visuals in different languages if you would like!

As this assignment is for the whole course, you must incorporate principles discussed in the first week, such as having as high data-ink ratio (Tufte) and aligning with Cairo’s principles of truth, beauty, function, and insight.

Here are the assignment instructions:

 * State the region and the domain category that your data sets are about (e.g., **Ann Arbor, Michigan, United States** and **economic activity or measures**).
 * You must state a question about the domain category and region that you identified as being interesting.
 * You must provide at least two links to available datasets. These could be links to files such as CSV or Excel files, or links to websites which might have data in tabular form, such as Wikipedia pages.
 * You must upload an image which addresses the research question you stated. In addition to addressing the question, this visual should follow Cairo's principles of truthfulness, functionality, beauty, and insightfulness.
 * You must contribute a short (1-2 paragraph) written justification of how your visualization addresses your stated research question.

What do we mean by **economic activity or measures**?  For this category you might look at the inputs or outputs to the given economy, or major changes in the economy compared to other regions.

## Tips
* Wikipedia is an excellent source of data, and I strongly encourage you to explore it for new data sources.
* Many governments run open data initiatives at the city, region, and country levels, and these are wonderful resources for localized data sources.
* Several international agencies, such as the [United Nations](http://data.un.org/), the [World Bank](http://data.worldbank.org/), the [Global Open Data Index](http://index.okfn.org/place/) are other great places to look for data.
* This assignment requires you to convert and clean datafiles. Check out the discussion forums for tips on how to do this from various sources, and share your successes with your fellow students!

## Example
Looking for an example? Here's what our course assistant put together for the **Ann Arbor, MI, USA** area using **sports and athletics** as the topic. [Example Solution File](./readonly/Assignment4_example.pdf)

In [ ]:

# Import useful libraries
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os

# Display all columns
pd.set_option('display.max_columns',None)

# Interactive matplotlib environment
%matplotlib notebook

# Set plot style
sns.set_style("white")

In [ ]:
# Read Spanish air quality data
spain_air = pd.read_csv('datasets/22.csv', encoding='latin8')

# Convert dates to datetime
spain_air['Fecha Solar (UTC)'] = pd.to_datetime(spain_air['Fecha Solar (UTC)'])

# Compute the mean for each hour of all the substations
spain_air = spain_air.groupby(['Fecha Solar (UTC)']).mean()

# Compute the mean measurements for every day
spain_air = spain_air.resample('D').mean().ix[:-1,:]

# Scale from ppm to 100ppm
spain_air['CO'] *= 100

# Select only the relevant columns, add measurement unit and unstack the pollutants
spain_air = (spain_air[['SO2', 'CO', 'NO2']].rename(columns={
            'SO2':'SO2 [ppb]', 'CO':'CO [10²ppm]', 
            'NO2':'NO2 [ppb]'}).unstack().reset_index(level=0)
            .rename(columns={'level_0': 'Pollutant', 0:'Value'}))

# Insert the name of the region
spain_air['Region'] = 'Gijón, Spain. Population: 0.273 million'

In [ ]:
# Find U.S.A air quality files
usa_air_files = os.listdir('datasets')
usa_air_files = [x for x in usa_air_files if 'daily' in x]

# Create name pollutants
indicators = ['SO2 [ppb]', 'CO [10²ppm]', 'NO2 [ppb]']

# Read dataframe for each pollutant and merge the measurements into one big dataset.
# Select only the measurements from New-York city
for i in range(3):
    temp = pd.read_csv('datasets/'+usa_air_files[i], usecols=['City Name', 'Date Local', 
                                                  'Arithmetic Mean'])
    temp = temp[temp['City Name']=='New York'].groupby(['Date Local']).mean()
    
    temp.rename(columns={'Arithmetic Mean': indicators[i]}, inplace=True)
    
    if i!=0:
        usa_air = usa_air.join(temp)
    else:
        usa_air = temp

# Scale from ppm to 100ppm
usa_air['CO [10²ppm]'] *= 100

# Unstack pollutants 
usa_air = usa_air.unstack().reset_index(level=0).rename(columns={'level_0': 'Pollutant', 0:'Value'})

# Insert region name
usa_air['Region'] = 'New York, U.S.A. Population: 8.491 million'

In [ ]:
# Concatenate the air quality measurements from both Gijon and New York
air_quality = pd.concat([spain_air, usa_air], axis=0)

# Draw a nested violinplot and split the violins for each region in part
plt.figure(figsize=(12,8))
sns.violinplot(x="Pollutant", y="Value", hue="Region", data=air_quality, split=True, inner='quartile');
plt.xlabel('Pollutant', fontsize=20, labelpad=20);
plt.ylabel('Emissions', fontsize=20);
plt.rc('xtick', labelsize=15);
plt.rc('ytick', labelsize=15);
plt.legend(loc='upper left', fontsize=13);
ttl = plt.title('Comparison of pollutant emissions in 2015 between Gijón and New York', fontsize=20)
ttl.set_position([.5, 1.02])